<a href="https://colab.research.google.com/github/utkarsh599/Recomendation-sysy/blob/Abhijeet/MicroVideoRecommandationSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-eu.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

In [17]:
!pip install py4j

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()

In [0]:
import pyspark

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
from pyspark.sql import SparkSession


In [0]:
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [0]:
videos1 =  spark.read.csv("/content/gdrive/My Drive/Colab Notebooks/GBvideos.csv", inferSchema=True, header=True)
videos2 =  spark.read.csv("/content/gdrive/My Drive/Colab Notebooks/USvideos.csv", inferSchema=True, header=True)

In [0]:
videos1=videos1.withColumn("LDdiff",videos1["likes"]-videos1["dislikes"])

In [32]:
videos1.show(5)

+-----------+--------------------+-----------------+-----------+--------------------+-------+-----+--------+-------------+--------------------+-----+------+
|   video_id|               title|    channel_title|category_id|                tags|  views|likes|dislikes|comment_total|      thumbnail_link| date|LDdiff|
+-----------+--------------------+-----------------+-----------+--------------------+-------+-----+--------+-------------+--------------------+-----+------+
|jt2OHQh0HoQ|Live Apple Event ...|      Apple Event|         28|apple events|appl...|7426393|78240|   13548|          705|https://i.ytimg.c...|13.09| 64692|
|AqokkXoa7uE|Holly and Phillip...|     This Morning|         24|this morning|inte...| 494203| 2651|    1309|            0|https://i.ytimg.c...|13.09|  1342|
|YPVcg45W0z4|My DNA Test Resul...|     emmablackery|         24|emmablackery|emma...| 142819|13119|     151|         1141|https://i.ytimg.c...|13.09| 12968|
|T_PuZBdT2iM|getting into a co...|            ProZD|      

In [0]:
videos2=videos2.withColumn("LDdiff",videos2["likes"]-videos2["dislikes"])

In [34]:
videos2.show(10)

+-----------+--------------------+--------------------+-----------+--------------------+-------+------+--------+-------------+--------------------+-----+------+
|   video_id|               title|       channel_title|category_id|                tags|  views| likes|dislikes|comment_total|      thumbnail_link| date|LDdiff|
+-----------+--------------------+--------------------+-----------+--------------------+-------+------+--------+-------------+--------------------+-----+------+
|XpVt6Z1Gjjo|1 YEAR OF VLOGGIN...|    Logan Paul Vlogs|         24|logan paul vlog|l...|4394029|320053|    5931|        46245|https://i.ytimg.c...|13.09|314122|
|K4wEI5zhHB0|iPhone X — Introd...|               Apple|         28|Apple|iPhone 10|i...|7860119|185853|   26679|            0|https://i.ytimg.c...|13.09|159174|
|cLdxuaxaQwc|         My Response|           PewDiePie|         22|              [none]|5845909|576597|   39774|       170708|https://i.ytimg.c...|13.09|536823|
|WYYvHb03Eog|Apple iPhone X fi...|

TypeError: ignored

In [0]:
videos1_rdd = videos1.select(['title']).rdd.flatMap(lambda x: x)
videos1_rdd = videos1_rdd.filter(lambda x: x is not None).filter(lambda x: x!='')
videos1_rdd.take(5)


videos2_rdd = videos2.select(['title']).rdd.flatMap(lambda x: x)
videos2_rdd = videos2_rdd.filter(lambda x: x is not None).filter(lambda x: x!='')
videos2_rdd.take(5)

['1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED YOUTUBE FOREVER!',
 'iPhone X — Introducing iPhone X — Apple',
 'My Response',
 'Apple iPhone X first look',
 'iPhone X (parody)']

In [0]:
header1_Vid=videos1_rdd.first()

header2_Vid=videos2_rdd.first()

In [0]:
data_rmv_col_vid1 = videos1_rdd.filter(lambda row: row != header1_Vid)

data_rmv_col_vid2 = videos2_rdd.filter(lambda row: row != header2_Vid)

In [0]:
def lower(x):
  return x.lower()


lowerCase1_VidRDD = data_rmv_col_vid1.map(lower)

lowerCase2_VidRDD = data_rmv_col_vid2.map(lower)

In [0]:
lowerCase1_VidRDD.take(5)

lowerCase2_VidRDD.take(5)

['iphone x — introducing iphone x — apple',
 'my response',
 'apple iphone x first look',
 'iphone x (parody)',
 'the disaster artist | official trailer hd | a24']

In [0]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [0]:
def sent_TokenizeFunct(x):
    doc=nlp(x)
    #return doc
    tokens=[]
    for token in doc:
      if((token.is_stop==False and token.is_punct==False) and token.is_alpha==True):
        tokens.append(token.lemma_)
    return ' '.join(tokens)

Vid_sentTokenizeRDD1 = lowerCase1_VidRDD.map(sent_TokenizeFunct)

Vid_sentTokenizeRDD2 = lowerCase2_VidRDD.map(sent_TokenizeFunct)

In [0]:
Vid_sentTokenizeRDD1.take(5)

['holly phillip meet samantha sex robot morning',
 'dna test result',
 'get conversation language actually speak',
 'baby challenge',
 'reveal fifa stat chelsea hazard luiz christensen']

In [0]:
Vid_sentTokenizeRDD2.take(5)

['iphone x introduce iphone x apple',
 'response',
 'apple iphone x look',
 'iphone x parody',
 'disaster artist official trailer hd']

In [0]:
tags_rdd1=videos1.select(['tags']).rdd.flatMap(lambda x:x)
tags_rdd2=videos2.select(['tags']).rdd.flatMap(lambda x:x)

In [0]:
header_tags1 = tags_rdd1.first()

In [0]:
tags_rmv_col1 = tags_rdd1.filter(lambda row: row != header_tags1)

In [0]:
def split1(x):
  return (x.split("|"))

In [0]:
tags_rdd1=tags_rmv_col1.map(split1)

In [0]:
from pyspark.sql import Row
from pyspark.sql.functions import monotonically_increasing_id

row_title1 = Row("LemmetizeTitle") # Or some other column name
Title_DF1 = Vid_sentTokenizeRDD1.map(row_title1).toDF()
Title_DF1 = Title_DF1.select("*").withColumn("id", monotonically_increasing_id())

row_tags1=Row("Tokenized_Tags")
Tags_DF1=tags_rdd1.map(row_tags1).toDF()
Tags_DF1 = Tags_DF1.select("*").withColumn("id", monotonically_increasing_id())

In [0]:
row_title2 = Row("LemmetizeTitle") # Or some other column name
Title_DF2 = Vid_sentTokenizeRDD2.map(row_title1).toDF()
Title_DF2 = Title_DF2.select("*").withColumn("id", monotonically_increasing_id())

row_tags2=Row("Tokenized_Tags")
Tags_DF2=tags_rdd2.map(row_tags2).toDF()
Tags_DF2 = Tags_DF2.select("*").withColumn("id", monotonically_increasing_id())

In [0]:
Title_DF2.show(5)

+--------------------+---+
|      LemmetizeTitle| id|
+--------------------+---+
|iphone x introduc...|  0|
|            response|  1|
| apple iphone x look|  2|
|     iphone x parody|  3|
|disaster artist o...|  4|
+--------------------+---+
only showing top 5 rows



In [0]:
Title_DF1.show(5)

+--------------------+---+
|      LemmetizeTitle| id|
+--------------------+---+
|holly phillip mee...|  0|
|     dna test result|  1|
|get conversation ...|  2|
|      baby challenge|  3|
|reveal fifa stat ...|  4|
+--------------------+---+
only showing top 5 rows



In [0]:
Tags_DF1.show(5)

+--------------------+---+
|      Tokenized_Tags| id|
+--------------------+---+
|[this morning, in...|  0|
|[emmablackery, em...|  1|
|[skit, korean, la...|  2|
|[sprinkleofglitte...|  3|
|[Rio Ferdinand, R...|  4|
+--------------------+---+
only showing top 5 rows



In [0]:
videos_DF1 = Title_DF1.join(Tags_DF1, on=['id'], how='left_outer')

In [0]:
videos_DF2 = Title_DF2.join(Tags_DF2, on=['id'], how='left_outer')

In [0]:
videos_DF1.show(5)

+----+--------------------+--------------------+
|  id|      LemmetizeTitle|      Tokenized_Tags|
+----+--------------------+--------------------+
|  26|testing soph x ma...|[grey hair, sophd...|
|  29|move vlog pink ho...|[moving vlog, ike...|
| 474|doom wolfenstein ...|[PC, IGN, PS4, Do...|
| 964|lloyd new lego ni...|[Google, YouTube,...|
|1677|project car ceo q...|[Project Cars 2 G...|
+----+--------------------+--------------------+
only showing top 5 rows



In [0]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer


tokenizer = Tokenizer(inputCol="LemmetizeTitle", outputCol="words")
wordsData = tokenizer.transform(videos_DF1)

hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=5)
featurizedData = hashingTF.transform(wordsData)

idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData1 = idfModel.transform(featurizedData)

rescaledData1.select("id", "features").show()

+----+--------------------+
|  id|            features|
+----+--------------------+
|  26|(5,[0,1,2,3,4],[0...|
|  29|(5,[0,1,2,4],[0.3...|
| 474|(5,[0,1,2,3],[0.7...|
| 964|(5,[0,1,4],[1.416...|
|1677|(5,[0,1,2,3,4],[0...|
|1697|(5,[0,1,2,4],[0.3...|
|1806|(5,[1,2,3,4],[0.3...|
|1950|(5,[1,3,4],[0.761...|
|2040|(5,[0,1],[1.06220...|
|2214|(5,[3,4],[0.44548...|
|2250|(5,[0,1,4],[0.354...|
|2453|(5,[0,2,3],[0.354...|
|2509|(5,[1,3,4],[1.142...|
|2529|(5,[0,2,4],[2.124...|
|2927|(5,[0,1,3],[0.354...|
|3091|(5,[0,1,2,3,4],[0...|
|3506|(5,[0,1,2,3,4],[1...|
|3764|(5,[0,4],[0.70813...|
|4590|(5,[0,1,4],[0.708...|
|4823|(5,[0,3,4],[0.354...|
+----+--------------------+
only showing top 20 rows



In [0]:
tokenizer = Tokenizer(inputCol="LemmetizeTitle", outputCol="words")
wordsData = tokenizer.transform(videos_DF2)

hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=5)
featurizedData = hashingTF.transform(wordsData)

idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData2 = idfModel.transform(featurizedData)

rescaledData2.select("id", "features").show()

+----+--------------------+
|  id|            features|
+----+--------------------+
|  26|(5,[0,1,2,3,4],[0...|
|  29|(5,[3,4],[0.47140...|
| 474|(5,[0,1,2,4],[0.7...|
| 964|(5,[0,1,3,4],[0.7...|
|1677|(5,[0,1],[0.35585...|
|1697|(5,[1,2,3,4],[0.3...|
|1806|(5,[0,3],[0.71170...|
|1950|(5,[1,2,4],[1.110...|
|2040|(5,[0,1,2,4],[0.7...|
|2214|(5,[0,1],[0.71170...|
|2250|(5,[0,3,4],[1.423...|
|2453|(5,[0,1,3,4],[0.3...|
|2509|(5,[0,4],[0.35585...|
|2529|(5,[0,1,4],[0.355...|
|2927|(5,[0,1,2,4],[0.7...|
|3091|(5,[0,1,2,3,4],[0...|
|3506|(5,[0,1,2],[0.711...|
|3764|(5,[0,1,2,3,4],[0...|
|4590|(5,[1,3,4],[0.370...|
|4823|(5,[1,2,3,4],[0.3...|
+----+--------------------+
only showing top 20 rows



In [0]:
from pyspark.ml.feature import MinHashLSH

MHL=MinHashLSH(inputCol="features", outputCol="lsh", numHashTables=5).fit(rescaledData1)
MHL.transform(rescaledData1).show()

+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|  id|      LemmetizeTitle|      Tokenized_Tags|               words|         rawFeatures|            features|                 lsh|
+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|  26|testing soph x ma...|[grey hair, sophd...|[testing, soph, x...|(5,[0,1,2,3,4],[1...|(5,[0,1,2,3,4],[0...|[[1.692739828E9],...|
|  29|move vlog pink ho...|[moving vlog, ike...|[move, vlog, pink...|(5,[0,1,2,4],[1.0...|(5,[0,1,2,4],[0.3...|[[1.692739828E9],...|
| 474|doom wolfenstein ...|[PC, IGN, PS4, Do...|[doom, wolfenstei...|(5,[0,1,2,3],[2.0...|(5,[0,1,2,3],[0.7...|[[1.742470823E9],...|
| 964|lloyd new lego ni...|[Google, YouTube,...|[lloyd, new, lego...|(5,[0,1,4],[4.0,3...|(5,[0,1,4],[1.416...|[[1.692739828E9],...|
|1677|project car ceo q...|[Project Cars 2 G...|[project, car, ce...|

In [0]:
from pyspark.sql.functions import col

In [0]:
print("Approximately joining dfA and dfB on distance smaller than 0.6:")
result=MHL.approxSimilarityJoin(rescaledData1, rescaledData2, 0.6, distCol="JaccardDistance")\
    .select(col("datasetA.id").alias("idA"),
            col("datasetB.id").alias("idB"),
            col("JaccardDistance"))

Approximately joining dfA and dfB on distance smaller than 0.6:


In [0]:
result.show()

+---+----+-------------------+
|idA| idB|    JaccardDistance|
+---+----+-------------------+
| 26|  89|0.19999999999999996|
| 26| 194|0.19999999999999996|
| 26| 369|0.19999999999999996|
| 26|1032|0.19999999999999996|
| 26|2662|                0.4|
| 26|2665|                0.4|
| 26|3223|0.19999999999999996|
| 26|3430|                0.4|
| 26|3790|                0.4|
| 26|3987|                0.0|
| 26|4687|                0.4|
| 26|5484|0.19999999999999996|
| 26|5916|0.19999999999999996|
| 26|6807|                0.4|
| 26|7275|                0.4|
| 26|7503|                0.4|
| 26|7552|0.19999999999999996|
| 26|7646|                0.4|
| 26|7780|0.19999999999999996|
| 29| 298|                0.4|
+---+----+-------------------+
only showing top 20 rows



In [0]:
from pyspark.sql.functions import countDistinct

In [0]:
from pyspark.sql.functions import rand
#df.agg(*(countDistinct(col(c)).alias(c) for c in df.columns))
result.orderBy(rand()).limit(120).show()

+----+----+-------------------+
| idA| idB|    JaccardDistance|
+----+----+-------------------+
|2927|2453|               0.25|
|4300|7151|                0.0|
|2281| 270|                0.0|
|6951|1015|0.33333333333333337|
|3845|4923|                0.0|
|6721|7382|                0.4|
|1280|2518|                0.0|
|1697|7310|                0.5|
| 130|5280|               0.25|
|3854|6440|0.33333333333333337|
|7358| 716|                0.5|
|3787|2691|                0.0|
| 185|6475|                0.5|
|4590|3310|                0.0|
|4072|7259|                0.5|
|7949| 413|                0.0|
| 558|1240|                0.4|
|3845|7900|                0.5|
|3280| 234|               0.25|
|3162|3017|                0.0|
+----+----+-------------------+
only showing top 20 rows

